In [ ]:
import pickle

In [ ]:
file = open("/content/sentences.pkl",'rb')
sentences = pickle.load(file)
file.close()

In [ ]:
file = open("/content/word_labels_list.pkl",'rb')
word_labels_list = pickle.load(file)
file.close()

In [ ]:
for id, sentence in enumerate(sentences):
  if len(sentence) < 4:
    del sentences[id]
    del word_labels_list[id]


In [ ]:
# function for converting lists into a pandas dataframe
import pandas as pd
def lists_to_dataframe(list1, list2, column_names=['Column1', 'Column2']):
    if len(list1) != len(list2):
        raise ValueError("Input lists must have the same length.")
    data = {column_names[0]: list1, column_names[1]: list2}
    df = pd.DataFrame(data)
    return df
# create a dataframe that contains a column of sentences and a column of the corresponding sentences' NER tags
data = lists_to_dataframe(sentences, word_labels_list, column_names=['sentence', 'word_labels'])
data

,sentence,word_labels
0,b robot hút bụi neato botvac d6 connected,"O, B-pn, I-pn, I-pn, O, O, O, O"
1,xin chào tôi muốn hỏi bạn về cách sửa lỗi cho ...,"O, O, O, O, O, O, O, O, O, O, O, O, B-pn, I-pn..."
2,bạn có thể làm mọi thứ bằng cả trái tim mình,"O, O, O, O, O, O, O, O, O, O, O"
3,7 với nhiều cảm biến thông minh robot hút bụi ...,"O, O, O, O, O, O, O, B-pn, I-pn, I-pn, O, O, O..."
4,1 tôi đã mua một máy lọc không khí từ cửa hàng...,"O, O, O, O, O, B-pn, I-pn, I-pn, I-pn, O, O, O..."
...,...,...
5197,Một trong những tính năng nổi bật của SUNTEK S...,"O,O,O,O,O,O,O,O,B-pn,I-pn,O,O,O,O,O,O,O,O,O,O,..."
5198,"Không chỉ vậy, SUNTEK SP-S30 còn được thiết kế...","O,O,O,B-pn,I-pn,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
5199,"Với SUNTEK SP-S30, bạn không chỉ tiết kiệm tiề...","O,B-pn,I-pn,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
5200,"Vậy còn chần chừ gì nữa, hãy sở hữu ngay SUNTE...","O,O,O,O,O,O,O,O,O,O,B-pn,I-pn,O,O,O,O,O,O,O,O,..."


In [ ]:
# remove spaces in the word_labels column
data['word_labels'] = data['word_labels'].str.replace(' ', '')
data

,sentence,word_labels
0,b robot hút bụi neato botvac d6 connected,"O,B-pn,I-pn,I-pn,O,O,O,O"
1,xin chào tôi muốn hỏi bạn về cách sửa lỗi cho ...,"O,O,O,O,O,O,O,O,O,O,O,O,B-pn,I-pn,I-pn,O,O,O,O..."
2,bạn có thể làm mọi thứ bằng cả trái tim mình,"O,O,O,O,O,O,O,O,O,O,O"
3,7 với nhiều cảm biến thông minh robot hút bụi ...,"O,O,O,O,O,O,O,B-pn,I-pn,I-pn,O,O,O,O,O,O,O,O,O..."
4,1 tôi đã mua một máy lọc không khí từ cửa hàng...,"O,O,O,O,O,B-pn,I-pn,I-pn,I-pn,O,O,O,O,O,O,O,O,..."
...,...,...
5197,Một trong những tính năng nổi bật của SUNTEK S...,"O,O,O,O,O,O,O,O,B-pn,I-pn,O,O,O,O,O,O,O,O,O,O,..."
5198,"Không chỉ vậy, SUNTEK SP-S30 còn được thiết kế...","O,O,O,B-pn,I-pn,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
5199,"Với SUNTEK SP-S30, bạn không chỉ tiết kiệm tiề...","O,B-pn,I-pn,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
5200,"Vậy còn chần chừ gì nữa, hãy sở hữu ngay SUNTE...","O,O,O,O,O,O,O,O,O,O,B-pn,I-pn,O,O,O,O,O,O,O,O,..."


In [ ]:
# assign NER tags numerical ids
label2id = {'I-pn': 2, 'B-pn': 1, 'O': 0}
id2label = {0 : 'O', 1 : 'B-pn', 2: 'I-pn'}

In [ ]:
# import libraries for training
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification, AdamW

In [ ]:
# check cuda
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
# configure training
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 10
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 3.0
# weight_decay = 0.01
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [ ]:
# set up BERT tokenizer
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
print(tokenizer.tokenize('AIOSMART AD200'))

['AI@@', 'OS@@', 'MA@@', 'RT', 'AD@@', '200']


In [ ]:
# define dataset class to pre-process dataset for training
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.sentence[index]
        word_labels = self.data.word_labels[index]
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)

        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]

        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [ ]:
# split dataset into training and testing
train_size = 0.7
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (5354, 2)
TRAIN Dataset: (3748, 2)
TEST Dataset: (1606, 2)


In [ ]:
# set up params and dataloaders
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
# load BERT model
model = AutoModelForTokenClassification.from_pretrained("vinai/phobert-base", num_labels=3,
                                                        id2label=id2label,
                                                        label2id=label2id)

model.to(device)

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (L

In [ ]:
# verify initial loss
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(1.3208, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:

# verify tensor shape
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 3])

In [ ]:
# set up optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
# start training
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_preds.extend(predictions)
        tr_labels.extend(targets)

        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

for epoch in range(EPOCHS):
  print(f"Training epoch: {epoch + 1}")
  train(epoch)

Training epoch: 1
Training loss per 100 training steps: 1.0632445812225342
Training loss per 100 training steps: 0.25850117837290953
Training loss per 100 training steps: 0.18643679249271825
Training loss per 100 training steps: 0.14759691887878598
Training loss per 100 training steps: 0.1233744379504586
Training loss per 100 training steps: 0.10738874361394884
Training loss per 100 training steps: 0.09577384467907683
Training loss per 100 training steps: 0.08725075091709636
Training loss per 100 training steps: 0.08007946081625789
Training loss per 100 training steps: 0.07475883251373407
Training loss epoch: 0.07275251317555592
Training accuracy epoch: 0.9309115216486044
Training epoch: 2
Training loss per 100 training steps: 0.058133307844400406
Training loss per 100 training steps: 0.025029321945742658
Training loss per 100 training steps: 0.02273651417483812
Training loss per 100 training steps: 0.022556668684270195
Training loss per 100 training steps: 0.022832229966297746
Trainin

In [ ]:
# evaluate trained model performance
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            print(predictions)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.0001687001349637285
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 0, 2], device='cuda:0')
tensor([0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0],
       device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
# Sau khi huấn luyện xong mô hình
# torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             }, 'model_checkpoint.pth')

model.save_pretrained("/content/model")
tokenizer.save_pretrained("/content/model")

('/content/model/tokenizer_config.json',
 '/content/model/special_tokens_map.json',
 '/content/model/vocab.txt',
 '/content/model/bpe.codes',
 '/content/model/added_tokens.json')

In [ ]:
def invert_tensor_to_label(sentence, predictions):
  token = tokenizer.tokenize(sentence)
  label_list = []
  if(len(predictions[0].tolist()) == len(sentence.split()) + 2):
    for idx,i in enumerate (predictions[0].tolist()):
      if i != 0:
        s = s + token[idx-1] + ' '
      else:
        if(predictions[0].tolist()[idx-1] !=0):
          label_list.append(s.strip())
          s = ''
  if(len(predictions[0].tolist()) > len(sentence.split()) + 2):
      i = 0
      tensor_list = predictions[0].tolist()
      # print(len(tensor_list))
      s = ''
      while i < len(tensor_list):
        # print(i)
        if tensor_list[i]!=0:
          if(tensor_list[i-1] == 2  and tensor_list[i] == 1):
            label_list.append(s.strip())
            s = ''
          if(token[i-1][-1] != '@'):
            s = s + token[i-1] + ' '
            # print(token[i-1])
          else:
            s = s + token[i-1][:-2]
            # print(token[i-1])
        else:
          if(tensor_list[i-1]!=0):
            label_list.append(s.strip())
            s = ''
        i = i + 1
  label_list = list(set(label_list))
  return label_list


In [ ]:
import torch
from transformers import BertTokenizer, BertForTokenClassification
from transformers import RobertaForTokenClassification, RobertaTokenizer
# # Khởi tạo tokenizer và mô hình
# tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
# model = AutoModelForTokenClassification.from_pretrained("vinai/phobert-base", num_labels=3)

# # Tải trạng thái của mô hình và optimizer từ file đã lưu
# checkpoint = torch.load('model_checkpoint.pth')
# model.load_state_dict(checkpoint['model_state_dict'])
# model.eval()  # Đặt mô hình vào chế độ dự đoán

# Tải mô hình và tokenizer đã lưu
tokenizer = AutoTokenizer.from_pretrained("/content/model")
model = AutoModelForTokenClassification.from_pretrained("/content/model", num_labels=3)

# Ví dụ câu văn cần dự đoán
sentence = "Bố mẹ tôi cần tìm một chiếc đèn mới thay thế chiếc đèn ở nhà. Điều kiện của nó là dùng năng lượng mặt trời hoặc tiết kiệm năng lượng. Người ta giới thiệu tôi sản phẩm bộ đèn led năng lượng mặt trời gắn liền tấm pin công suất: 90W. Liệu bộ đèn led này có tốt không?"
inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Dự đoán
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# Lấy các dự đoán
predictions = torch.argmax(logits, dim=-1)
print(predictions)
print(invert_tensor_to_label(sentence, predictions))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0]])
['đèn', 'bộ đèn led', 'bộ đèn led năng lượng mặt trời gắn liền tấm pin công suất: 90W.']


In [ ]:
# Ví dụ câu văn cần dự đoán
sentence = "Bạn x có tố chất thông minh cùng với sự cố gắng không ngừng nghỉ, bạn đã giành giải nhất môn toán. Trong không khí vui mừng chiến thắng, bạn đã được bố mẹ thưởng cho một con robot đồ chơi và bàn ủi khô bluestone DIB-3776 1300W"
inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Dự đoán

with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# Lấy các dự đoán
predictions = torch.argmax(logits, dim=-1)
print(predictions)
print(invert_tensor_to_label(sentence, predictions))

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0]])
['bàn ủi khô bluestone DIB-3776 1300W']


In [ ]:
# Ví dụ câu văn cần dự đoán
sentence = """Bạn x có tố chất thông minh cùng với sự cố gắng không ngừng nghỉ, bạn đã giành giải nhất môn toán. Trong không khí vui mừng chiến thắng, bạn đã được
bố mẹ thưởng cho một con robot đồ chơi và bàn ủi khô bluestone DIB-3776 1300W. Bàn ủi là thiết bị cần có ở mọi nhà.
Bên cạnh đó, Kalite KL-1500 là một nồi chiên không dầu rất tốt, gia đình đang cân nhắc mua"""
inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Dự đoán

with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# Lấy các dự đoán
predictions = torch.argmax(logits, dim=-1)
print(predictions)
print(invert_tensor_to_label(sentence, predictions))

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 1, 2, 2, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
96
['bàn ủi khô bluestone DIB-3776 1300W.', 'Bàn ủi', 'Kalite KL-1500', 'nồi chiên không dầu']


In [ ]:
# Ví dụ câu văn cần dự đoán
sentence = "Tôi mới làm mất đèn đường năng lượng mặt trời AIOSMART AD200. Vì vậy, tôi muốn đi tìm đèn mới"
inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Dự đoán

with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# Lấy các dự đoán
predictions = torch.argmax(logits, dim=-1)
print(predictions)
print(invert_tensor_to_label(sentence, predictions))

tensor([[0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0]])
['đèn đường năng lượng mặt trời AIOSMART AD200.', 'đèn']


In [ ]:
predictions[0].tolist()
label_list = []
s = ''
token = tokenizer.tokenize(sentence)
print(token)
# for idx,i in enumerate (predictions[0].tolist()):
#   if i != 0:
#     s = token[idx-1] + ' '
#   else:
#     if()
#     label_list.append(s.strip())
#     s = ''
# label_list

['Tôi', 'mới', 'làm', 'mất', 'đèn', 'đường', 'năng', 'lượng', 'mặt', 'trời', 'AI@@', 'OS@@', 'MA@@', 'RT', 'AD@@', '200@@', '.', 'Vì', 'vậ@@', 'y@@', ',', 'tôi', 'muốn', 'đi', 'tìm', 'đèn', 'mới']


In [ ]:
token = tokenizer.tokenize('kasdamk djasnjda')
token

['kas@@', 'dam@@', 'k', 'd@@', 'j@@', 'as@@', 'n@@', 'j@@', 'da']

In [ ]:
sentence

'máy giặt nào tốt'

In [ ]:
len(sentence.split())

4

In [ ]:
token

['máy', 'giặt', 'nào', 'tốt']

In [ ]:
token = tokenizer.tokenize(sentence)
if(len(predictions[0].tolist()) == len(token) + 2):
  for idx,i in enumerate (predictions[0].tolist()):
    if i != 0:
      s = s + token[idx-1] + ' '
    else:
      if(predictions[0].tolist()[idx-1] !=0):
        label_list.append(s.strip())
        s = ''
  label_list

In [ ]:
label_list = []

In [ ]:
token

['Tôi',
 'mới',
 'làm',
 'mất',
 'đèn',
 'đường',
 'năng',
 'lượng',
 'mặt',
 'trời',
 'AI@@',
 'OS@@',
 'MA@@',
 'RT',
 'AD@@',
 '200@@',
 '.',
 'Vì',
 'vậ@@',
 'y@@',
 ',',
 'tôi',
 'muốn',
 'đi',
 'tìm',
 'đèn',
 'mới']

In [ ]:
sentence = "Tôi mới làm mất đèn đường năng lượng mặt trời AIOSMART AD200. Vì vậy, tôi muốn đi tìm đèn mới"

In [ ]:
label_list = []

In [ ]:
sentence = "Bạn x có tố chất thông minh cùng với sự cố gắng không ngừng nghỉ, bạn đã giành giải nhất môn toán. Trong không khí vui mừng chiến thắng, bạn đã được bố mẹ thưởng cho một con robot đồ chơi và bàn ủi khô bluestone DIB-3776 1300W. Bàn ủi là thiết bị cần có ở mọi nhà. Bên cạnh đó, Kalite KL-1500 là một nồi chiên không dầu rất tốt, gia đình đang cân nhắc mua"
token = tokenizer.tokenize(sentence)
# sentence = "Bạn x có tố chất thông minh cùng với sự cố gắng không ngừng nghỉ, bạn đã giành giải nhất môn toán. Trong không khí vui mừng chiến thắng, bạn đã được bố mẹ thưởng cho một con robot đồ chơi và bàn ủi khô bluestone DIB-3776 1300W"

In [ ]:
print(len(token))

94


In [ ]:
label_list = []

In [ ]:
!zip -r '/content/model_ner.zip' '/content/model'

  adding: content/model/ (stored 0%)
  adding: content/model/added_tokens.json (stored 0%)
  adding: content/model/special_tokens_map.json (deflated 57%)
  adding: content/model/vocab.txt (deflated 55%)
  adding: content/model/model.safetensors (deflated 18%)
  adding: content/model/config.json (deflated 51%)
  adding: content/model/bpe.codes (deflated 59%)
  adding: content/model/tokenizer_config.json (deflated 77%)
